## Login Connection For DataRobot

In [8]:
import pandas as pd
import datarobot as dr
import numpy as np
#Make sure that you are running with the latest version
print(dr.__version__)

with open (r"G:\DataRobot\JNDataRobotAPIKey.txt", "r") as myfile:
    VPNToken=myfile.readlines()
    
#Fill in the API token.You can get the
API_TOKEN = VPNToken
END_POINT = 'https://datarobot.factset.com/api/v2'

#Instantiate the client 
dr.Client(token= API_TOKEN, endpoint=END_POINT)

2.19.0


## Browse Project List

In [9]:
#See theat you can navigate the current projects
current_projects  = dr.Project.list()
current_projects

[Project(API - DR - Data Export - 10.csv),
 Project(Data Export - All 10B Yield3 Payout90 Growth0.csv),
 Project(Data Export - All 5B Yield3 Payout90 Growth0.csv),
 Project(Data Export - All 1B Yield3 Payout90 Growth0.csv),
 Project(Data Export All 10B.csv),
 Project(Data Export All 100M.csv),
 Project(Data Export - All 100M Yield3 Payout90 Growth0.csv),
 Project(All Top Half - 12M Classification),
 Project(All Top Half - 3M Classification),
 Project(All Top Half - 1M Classification)]

In [23]:
#Clone a project
project_to_clone = current_projects[0].clone_project(new_project_name='Cloned Project')
project = project_to_clone

#Show a select
project = current_projects[0]
models = project.get_datetime_models()
models
#Select a model
model = models[0]
vars(model)

{'id': '5e489a68d94a070ca21e2dbd',
 'processes': ['Missing Values Imputed', 'RandomForest Classifier (Gini)'],
 'featurelist_name': 'DR Reduced Features M6',
 'featurelist_id': '5e489a40d94a070c611e2db1',
 'project_id': '5e48974bc08c300516b56215',
 'sample_pct': None,
 'training_row_count': None,
 'training_duration': None,
 'training_start_date': datetime.datetime(2011, 12, 30, 0, 0, tzinfo=tzutc()),
 'training_end_date': datetime.datetime(2020, 2, 8, 0, 0, tzinfo=tzutc()),
 'model_type': 'RandomForest Classifier (Gini)',
 'model_category': 'model',
 'is_frozen': True,
 'blueprint_id': '26f4b08053cf64b1976a3b4312bc6239',
 'metrics': {'AUC': {'backtesting': None,
   'holdout': None,
   'backtestingScores': [None, None, None, None, None],
   'crossValidation': None,
   'validation': None},
  'Rate@Top5%': {'backtesting': None,
   'holdout': None,
   'backtestingScores': [None, None, None, None, None],
   'crossValidation': None,
   'validation': None},
  'Gini Norm': {'backtesting': Non

In [20]:
#See feature lists
feature_lists = project.get_featurelists()
feature_lists[2].id

'5e489769c08c300507b561da'

In [5]:
#see features
raw_features = [lst for lst in feature_lists if lst.name == 'Raw Features'][0]
#raw_features.features

#see used features
used_features = [lst for lst in feature_lists if lst.name == 'DR Reduced Features M6'][0]
used_features.features

['Payout EPS',
 'B/P',
 'B/P ex Goodwill',
 'Div Yld',
 'PEG',
 'S/P',
 'Tang B/P',
 'Gross Margin',
 'Net Profit Margin',
 'Pretax Margin',
 'ROE',
 'Return Tot Cap',
 'Return Tang Common Eq',
 'CapEx to Assets',
 'CF Coverage Ratio',
 'CFO Ratio',
 'Total Debt Ratio',
 'Debt to Com Eq',
 'Debt to Eq',
 'Financial Leverage Ratio',
 'Quick Ratio',
 'CCC',
 'Eq Turnover Ratio',
 'Fixed Asset Turnover Ratio',
 'Inventory Turnover Ratio',
 'Payables Turnover Ratio',
 'Total Asset Turnover Ratio',
 '5Y EPS Grow',
 '1Y DPS Grow',
 '5Y DPS Grow',
 '5Y Sales Grow',
 '5Y Beta SP50',
 'SUE Ann',
 '1Y Price Vol',
 '3Yr Price Vol',
 'FCF Yield',
 '1 Year Growth Divs Per Sh',
 'FE Val Payout Mean FY1 Roll',
 'MktVal Co',
 'Outperform 12_Month']

## Cloning a Project

In [ ]:

new_project = project.clone_project(new_project_name='Cloned Project')
new_projet.name
new_project.id != project.id


## Create New Project

In [11]:
#Define few global variables here
file_name = 'DR - Data Export - 10.csv'
TRAINING_FILE_PATH = r'G:\Python Data\FactSet Alpha Testing Export'
data_raw_file = pd.read_csv(fr'G:\Python Data\FactSet Alpha Testing Export\{file_name}')
project_name = f'API - {file_name}'

#Export all columns to a text file
all_columns = pd.DataFrame(data_raw_file.columns)
all_columns.to_csv(r'G:\Python Data\FactSet Alpha Testing Export\API - All Columns.csv',index=False)


In [ ]:
#Create the time based project in DataRobot
project = dr.Project.create(data_raw_file, project_name='New Goal')

## Set up features 

In [ ]:
#Read in the list of features to remove
features_to_remove = open(r'G:\Python Data\FactSet Alpha Testing Export\API - FeaturesToRemove.csv','r').read().split('\n')
features = project.get_features()
pd.DataFrame([f.__dict__ for f in features])

#save down raw features
raw_features = [lst for lst in feature_lists if lst.name == 'Raw Features'][0]

## Special handling for Time Based Projects 
no_forward_features_list = list(set(raw_features.features) - set(features_to_remove))

#This creates new featurelist. This will throw an error if the featurelist exists
new_features_no_forward_feature_list = project.create_featurelist('no_forward_features_list', no_forward_features_list)
no_forward_feature_list_id = new_features_no_forward_feature_list.id

Setup Time Aware Modelling

In [15]:
#It is important to choose the correct dateTime column (exact match)
spec = dr.DatetimePartitioningSpecification('Period (YYYYMMDD)')
partitioning_preview = dr.DatetimePartitioning.generate(project.id, spec)
print(partitioning_preview.to_dataframe())
spec.number_of_backtests = 5
partitioning_preview = dr.DatetimePartitioning.generate(project.id, spec)
print(partitioning_preview.to_dataframe())


ClientError: 422 client error: {'message': 'Cannot change the date format of any feature after setting the target'}

## Run Autopilot

In [21]:
#This returns after starting to build models. This is asynchronous call and one has to make
# a separate call to finish the project
target_name = r'Outperform 6-Month'
project.set_worker_count(-1)
project.set_target(target_name, partitioning_method=spec, featurelist_id=feature_lists[2].id)
project

ClientError: 422 client error: {'message': 'Target Outperform 12_Month was already selected.'}